# TFM. Modelo predictivo llamadas Call Center
## MVP

## Importacion de librerias 

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import datetime
import calendar

pd.options.display.max_columns = None

### Importacion de los datos de envios por oficinas

In [2]:
df_shippings = pd.read_csv('data/shippings.csv', low_memory=False)

In [3]:
df_shippings['Franquicia destino'].unique()

array([4800, 4802, 4803, 4806, 4810])

##### Filtramos oficinas

In [4]:
#df = df_shippings[ (df_shippings['Franquicia destino']==4800) ]

In [5]:
df = df_shippings
df.columns

Index(['Unnamed: 0', 'Fecha envío', 'Número envío', 'Id. Fiscal',
       'Nombre Comercial', 'Código servicio', 'Nombre Rem', 'Población Rem',
       'CP Rem', 'Nombre vía Rem', 'Nombre', 'Población', 'Código postal',
       'Nombre vía', 'Total bultos', 'Franquicia origen', 'Franquicia destino',
       'Total', 'Estado', 'Tipo anomalía', 'Motivo', 'Importe Total'],
      dtype='object')

In [6]:
df = df.iloc[:, [1,5,12,14,15,16]]
df.head()

,Fecha envío,Código servicio,Código postal,Total bultos,Franquicia origen,Franquicia destino
0,01/01/2020,Urgente 12,07011,1,636,4800
1,01/01/2020,Económico,07003,2,640,4800
2,01/01/2020,Económico,07001,1,642,4800
3,01/01/2020,Urgente 10,07012,1,679,4800
4,01/01/2020,E-commerce,07003,1,2638,4800


In [7]:
df['Fecha envío'] = pd.to_datetime(df['Fecha envío'])

/home/dsc/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


#### Filtramos fechas en el dataset

In [8]:
#df = df[df['Fecha envío'] > '01-6-2020']
#df.head()

In [9]:
def updateDate(s):
    envio24h = ['Urgente 8:30', 'Urgente 10','Urgente 12','Urgente 14','Urgente 19', \
               'Urgente 8:30 expedición', 'Urgente 10 expedición','Urgente 12 expedición', \
               'Urgente 14 expedición','Urgente 19 expedición','Urgente hoy','Burofax', \
               'Documentos 14','Mascotas','Bag 14','Bag 19','Express documentos']
    envio48h = ['Económico','E-commerce','Mrw economy (ecopaq)','Retorno canje']
    maritimo = ['Marítimo baleares']
    
    nDate = np.nan
    ## Arreglar los 24H
    if s['Código servicio'] in envio24h:
        nDate = s['Fecha envío'] + datetime.timedelta(days=0)
    elif s['Código servicio'] in envio48h:
        nDate = s['Fecha envío'] + datetime.timedelta(days=0)
    else:
        nDate = s['Fecha envío'] + datetime.timedelta(days=0)
    
    if nDate.isoweekday() == 7:
        nDate = nDate + datetime.timedelta(days=0)
    elif nDate.isoweekday() == 6:
        nDate = nDate + datetime.timedelta(days=0)
        
    return nDate

In [10]:
df['xDate'] = df.apply(updateDate, axis=1)

/home/dsc/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
df[df['xDate'].isnull()]

,Fecha envío,Código servicio,Código postal,Total bultos,Franquicia origen,Franquicia destino,xDate


In [12]:
df['Fecha envío'] =  pd.to_datetime(df['xDate'])

/home/dsc/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
dias = list(calendar.day_name)

In [14]:
df['dayofweek'] = [dias[numero] for numero in df['Fecha envío'].dt.dayofweek]
df.head()

/home/dsc/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Fecha envío,Código servicio,Código postal,Total bultos,Franquicia origen,Franquicia destino,xDate,dayofweek
0,2020-01-01,Urgente 12,07011,1,636,4800,2020-01-01,Wednesday
1,2020-01-01,Económico,07003,2,640,4800,2020-01-01,Wednesday
2,2020-01-01,Económico,07001,1,642,4800,2020-01-01,Wednesday
3,2020-01-01,Urgente 10,07012,1,679,4800,2020-01-01,Wednesday
4,2020-01-01,E-commerce,07003,1,2638,4800,2020-01-01,Wednesday


In [15]:
df['Fecha envío'] =  pd.to_datetime(df['Fecha envío']).astype('str')

/home/dsc/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
df['Código servicio'].unique()

array(['Urgente 12', 'Económico', 'Urgente 10', 'E-commerce',
       'Urgente 19', 'Mascotas', 'Marítimo baleares', 'Urgente 14',
       'Urgente 19 expedición', 'Urgente 12 expedición', 'Urgente 8:30',
       'Burofax', 'Retorno albarán', 'Bag 14', 'Urgente 10 expedición',
       'Urgente hoy', 'Documentos 14', 'Bag 19',
       'Urgente 8:30 expedición', 'Mrw economy (ecopaq)',
       'Urgente 14 expedición', 'Express documentos', 'Retorno canje',
       'Urgente funchal', 'Express paquetes', 'Devolución burofax',
       'E-Commerce', 'Plan ayuda', 'Ecobox 25',
       'OperativaServicioEnumECOBA', 'E-commerce canje',
       'Retorno burofax'], dtype=object)

In [17]:
train = df[['Fecha envío','Código servicio','Franquicia origen','Código postal','dayofweek']]
train.reset_index(inplace=True)
train = train[['Fecha envío','Código servicio','Franquicia origen','Código postal','dayofweek']]
train['total'] = 1

In [18]:
train.head()

,Fecha envío,Código servicio,Franquicia origen,Código postal,dayofweek,total
0,2020-01-01,Urgente 12,636,07011,Wednesday,1
1,2020-01-01,Económico,640,07003,Wednesday,1
2,2020-01-01,Económico,642,07001,Wednesday,1
3,2020-01-01,Urgente 10,679,07012,Wednesday,1
4,2020-01-01,E-commerce,2638,07003,Wednesday,1


In [19]:
train['Franquicia origen'] = train['Franquicia origen'].apply(str)
train['Código postal'] = train['Código postal'].apply(str)

In [20]:
encoder = OneHotEncoder(handle_unknown='ignore')
enc = train[['Código servicio']]

In [21]:
encoder.fit(enc.values.reshape(-1,1))
encoder.categories_

[array(['Bag 14', 'Bag 19', 'Burofax', 'Devolución burofax',
        'Documentos 14', 'E-Commerce', 'E-commerce', 'E-commerce canje',
        'Ecobox 25', 'Económico', 'Express documentos', 'Express paquetes',
        'Marítimo baleares', 'Mascotas', 'Mrw economy (ecopaq)',
        'OperativaServicioEnumECOBA', 'Plan ayuda', 'Retorno albarán',
        'Retorno burofax', 'Retorno canje', 'Urgente 10',
        'Urgente 10 expedición', 'Urgente 12', 'Urgente 12 expedición',
        'Urgente 14', 'Urgente 14 expedición', 'Urgente 19',
        'Urgente 19 expedición', 'Urgente 8:30', 'Urgente 8:30 expedición',
        'Urgente funchal', 'Urgente hoy'], dtype=object)]

In [22]:
encoded = encoder.transform(enc.values.reshape(-1,1))
encoded_df = pd.DataFrame(encoded.todense())
encoded_df.columns = encoder.categories_[0]

In [23]:
train = pd.concat([train,encoded_df], axis=1)

In [24]:
#encoder = OneHotEncoder(handle_unknown='ignore')
enc = train[['dayofweek']]
encoder.fit(enc.values.reshape(-1,1))
encoded = encoder.transform(enc.values.reshape(-1,1))
encoded_df = pd.DataFrame(encoded.todense())
encoded_df.columns = encoder.categories_[0]

In [25]:
train = pd.concat([train,encoded_df], axis=1)

In [26]:
#encoder = OneHotEncoder(handle_unknown='ignore')
#enc = train[['Código postal']]
#encoder.fit(enc.values.reshape(-1,1))
#encoded = encoder.transform(enc.values.reshape(-1,1))
#encoded_df = pd.DataFrame(encoded.todense())
#encoded_df.columns = encoder.categories_[0]

In [27]:
#train = pd.concat([train,encoded_df], axis=1)

In [28]:
train.drop(columns=['Código servicio','Franquicia origen','Código postal','dayofweek'], inplace=True)

In [29]:
train = train.groupby(['Fecha envío']).sum()

In [30]:
train.head()

,total,Bag 14,Bag 19,Burofax,Devolución burofax,Documentos 14,E-Commerce,E-commerce,E-commerce canje,Ecobox 25,Económico,Express documentos,Express paquetes,Marítimo baleares,Mascotas,Mrw economy (ecopaq),OperativaServicioEnumECOBA,Plan ayuda,Retorno albarán,Retorno burofax,Retorno canje,Urgente 10,Urgente 10 expedición,Urgente 12,Urgente 12 expedición,Urgente 14,Urgente 14 expedición,Urgente 19,Urgente 19 expedición,Urgente 8:30,Urgente 8:30 expedición,Urgente funchal,Urgente hoy,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
Fecha envío,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-01,99,0.0,0.0,0.0,0.0,0.0,0.0,67.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,21.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99.0
2020-01-02,704,0.0,0.0,0.0,0.0,0.0,0.0,92.0,0.0,0.0,32.0,0.0,0.0,152.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,6.0,1.0,0.0,0.0,417.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,704.0,0.0,0.0
2020-01-03,193,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,186.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,193.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-04,1480,0.0,1.0,1.0,0.0,0.0,0.0,396.0,0.0,0.0,142.0,0.0,0.0,595.0,6.0,0.0,0.0,0.0,1.0,0.0,0.0,64.0,1.0,156.0,15.0,0.0,0.0,80.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,1480.0,0.0,0.0,0.0,0.0
2020-01-05,134,0.0,0.0,0.0,0.0,0.0,0.0,23.0,0.0,0.0,4.0,0.0,0.0,97.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,134.0,0.0,0.0,0.0


In [31]:
train.columns

Index(['total', 'Bag 14', 'Bag 19', 'Burofax', 'Devolución burofax',
       'Documentos 14', 'E-Commerce', 'E-commerce', 'E-commerce canje',
       'Ecobox 25', 'Económico', 'Express documentos', 'Express paquetes',
       'Marítimo baleares', 'Mascotas', 'Mrw economy (ecopaq)',
       'OperativaServicioEnumECOBA', 'Plan ayuda', 'Retorno albarán',
       'Retorno burofax', 'Retorno canje', 'Urgente 10',
       'Urgente 10 expedición', 'Urgente 12', 'Urgente 12 expedición',
       'Urgente 14', 'Urgente 14 expedición', 'Urgente 19',
       'Urgente 19 expedición', 'Urgente 8:30', 'Urgente 8:30 expedición',
       'Urgente funchal', 'Urgente hoy', 'Friday', 'Monday', 'Saturday',
       'Sunday', 'Thursday', 'Tuesday', 'Wednesday'],
      dtype='object')

In [32]:
train.describe().iloc[0,0] ##Cantidad de registros

399.0

In [33]:
columnas = ['E-commerce','Económico','Marítimo baleares',\
            'Urgente 10','Urgente 12','Urgente 19','Urgente 19 expedición',\
            'Monday','Tuesday','Wednesday','Thursday','Friday']

In [34]:
#train = train[columnas]
#train.head()

In [35]:
#train = train.iloc[:,1:-7]

In [36]:
#train.head()

In [37]:

train.Monday = [ 1 if value > 0 else 0 for value in train.Monday]
train.Tuesday = [ 1 if value > 0 else 0 for value in train.Tuesday]
train.Wednesday = [ 1 if value > 0 else 0 for value in train.Wednesday]
train.Thursday = [ 1 if value > 0 else 0 for value in train.Thursday]
train.Friday = [ 1 if value > 0 else 0 for value in train.Friday]
train.Saturday = [ 1 if value > 0 else 0 for value in train.Saturday]
train.Sunday = [ 1 if value > 0 else 0 for value in train.Sunday]
train.head()


,total,Bag 14,Bag 19,Burofax,Devolución burofax,Documentos 14,E-Commerce,E-commerce,E-commerce canje,Ecobox 25,Económico,Express documentos,Express paquetes,Marítimo baleares,Mascotas,Mrw economy (ecopaq),OperativaServicioEnumECOBA,Plan ayuda,Retorno albarán,Retorno burofax,Retorno canje,Urgente 10,Urgente 10 expedición,Urgente 12,Urgente 12 expedición,Urgente 14,Urgente 14 expedición,Urgente 19,Urgente 19 expedición,Urgente 8:30,Urgente 8:30 expedición,Urgente funchal,Urgente hoy,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
Fecha envío,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-01,99,0.0,0.0,0.0,0.0,0.0,0.0,67.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,21.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,1
2020-01-02,704,0.0,0.0,0.0,0.0,0.0,0.0,92.0,0.0,0.0,32.0,0.0,0.0,152.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,6.0,1.0,0.0,0.0,417.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0
2020-01-03,193,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,186.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0
2020-01-04,1480,0.0,1.0,1.0,0.0,0.0,0.0,396.0,0.0,0.0,142.0,0.0,0.0,595.0,6.0,0.0,0.0,0.0,1.0,0.0,0.0,64.0,1.0,156.0,15.0,0.0,0.0,80.0,22.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0
2020-01-05,134,0.0,0.0,0.0,0.0,0.0,0.0,23.0,0.0,0.0,4.0,0.0,0.0,97.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0,0,0


In [38]:
train = train.drop(labels=['Saturday','Sunday'], axis=1)

In [39]:
train.to_pickle('./data/df_Envios')